In [1]:
import tracemalloc

from utils_v1 import *


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates,
    PA, PADeforestationTime, PADistanceMap, PAMultipleDates,
    MTDeforestationTime,
    MTMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential

from src.trainer.evidential import TrainerEvidential

from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.trainer.evidential as evidential


In [3]:
'''
if config["training"] == True:
	dates = [2017, 2018]
else:
	dates = [2018, 2019]
if config["training"] == True:
	dates = [2018, 2019]
else:
	dates = [2019, 2020]
'''


#dates = [2018, 2019]
dates = [2017, 2018]

dataset = PAMultipleDates(dates = dates, # dates = [2016,2017,2018,2019]
	addPastDeforestationInput = True,
	borderBuffer = 2)	


'''
dates = [2019, 2020]

dataset = MTMultipleDates(dates = dates, # dates = [2016,2017,2018,2019]
	addPastDeforestationInput = True,
	borderBuffer = 2)	
'''

ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerEvidential(
		dataset)
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| dates: [2017, 2018]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [4]:
repetition_n = 10
experiment_values = np.ones(repetition_n).astype(np.uint8)


# experiment_values = [int(x) for x in experiment_values]
results = []

#exp_id_base = 148
# exp_id_base = 155
exp_id_base = 194
exp_id_base = 198
exp_id_base = 214

# earlier date
exp_id_base = 224

# MT
# exp_id_base = 196


exp_ids = range(exp_id_base, exp_id_base+len(experiment_values))


In [5]:
exp_ids

range(224, 234)

In [6]:
from enum import Enum
UncertaintyMethod = Enum('Predictor', 'pred_var MI pred_entropy KL pred_entropy_single evidential')
uncertaintyMethod = "evidential"


In [7]:
idx = 0
while idx < len(exp_ids):
# for experiment_value, exp_id in zip(experiment_values, exp_ids):
    try:
        config = {
        "training": True,
        "learning_rate": 1e-4,
        "weights": [0.1, 0.9, 0],
        "save_probabilities": False,
        "loadInference": False,
        "addPastDeforestationInput": True,
        "inference_times": 1,
        "Uncertainty": {
            "annealing_step": 10*375 # 1875
        },
        "uncertainty_method": uncertaintyMethod,
        "evidence_function": evidential.relu_evidence
        }
        print("Beginning run number {}".format(idx))
        trainer = TrainerEvidential(config, dataset, patchesHandler)

        trainer.defineExperiment(exp_ids[idx])

        trainer.setExperimentPath()
        trainer.createLogFolders()

        trainer.loadDataset()

        # %%
        if config["training"] == True:
            trainer.run()

        # %%
        result = trainer.run_predictor()
        print("Result idx {}: {}".format(idx, result))
        results.append(result)


        with open('log.pkl', 'wb') as f:
            pickle.dump(results, f)
        idx += 1
        print("Grid execution idx: {}".format(idx))
    except Exception as e: 
        print(e)
        print("Execution error in idx {}. Retrying...".format(idx))
        continue

Beginning run number 0
D:/Jorge/datasets/deforestation/Para/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Label where deforestation past years is actual date (2018) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

Epoch 1/100
self.global_step 0.0
alpha Tensor("loss/add:0", shape=(None, 128, 128, 3), dtype=float32)
u Tensor("loss/truediv:0", shape=(None, 128, 128, 1), dtype=float32)
A Tensor("loss/Sum_3:0", shape=(None, 3), dtype=float32)
B Tensor("loss/Sum_4:0", shape=(None, 1, 1, 3), dtype=float32)
term1 Tensor("loss/truediv_7:0", shape=(None, 1, 1, 3), dtype=float32)
term2 Tensor("loss/mul_5:0", shape=(None, 1, 1, 3), dtype=float32)
Tensor("loss/Repeat_1/Reshape_1:0", shape=(None, None, None, 1), dtype=float32)
alpha Tensor("add:0", shape=(None, 128, 128, 3), dtype=float32)
u Tensor("truediv:0", shape=(None, 128, 128, 1), dtype=float32)
K.int_shape(KL_reg) Tensor("mul_6:0", shape=(None, 128, 128, 1), dtype=float32)
alpha Tensor("add_7:0", shape=(None, 128, 128, 3), dtype=float32)
u Tensor("truediv_5:0", shape=(None, 128, 128, 1), dtype=float32)
K.int_shape(KL_reg) Tensor("mul_12:0", shape=(None, 128, 128, 1), dtype=float32)
alpha Tensor("add_16:0", shape=(None, 128, 128, 3), dtype=float32)
u T

c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


315/315 [==============================] - 30s 97ms/step - loss: 0.8193 - accuracy: 0.8699 - KL_term: 0.0158 - loglikelihood_term: 0.2175 - evidential_success: 27.4840 - evidential_fail: 8.0182 - acc: 0.8698 - annealing_coef: 0.0837 - global_step_get: 314.0000 - annealing_step_get: 3750.0000 - UEO_term: 0.3646 - val_loss: 0.9157 - val_accuracy: 0.6589 - val_KL_term: 0.0513 - val_loglikelihood_term: 0.5098 - val_evidential_success: 15.8682 - val_evidential_fail: 3.4930 - val_acc: 0.6589 - val_annealing_coef: 0.1675 - val_global_step_get: 628.0000 - val_annealing_step_get: 3750.0000 - val_UEO_term: 0.6303

Epoch 00002: val_loss did not improve from 0.87350
Epoch 3/100
self.global_step 628.0
315/315 [==============================] - 30s 97ms/step - loss: 0.8142 - accuracy: 0.8854 - KL_term: 0.0174 - loglikelihood_term: 0.1937 - evidential_success: 33.6106 - evidential_fail: 6.9578 - acc: 0.8855 - annealing_coef: 0.1256 - global_step_get: 471.0000 - annealing_step_get: 3750.0000 - UEO_ter

ic| self.path_models+ '/' + self.method +'_'+str(0)+'.h5': 'D:/Jorge/datasets/deforestation/experiments/exp224/models/resunet_0.h5'


time:  0


In [ ]:
print(results)

[{'other_uncertainty_metrics': {'sUEO': 0.3670126015642983, 'ece_score': 0.04906518334555479}, 'uncertainty_result': {'metrics': {'precision_L': array([0.8460478]), 'recall_L': array([0.92707223]), 'recall_Ltotal': array([0.55478324]), 'AA': array([0.03000072]), 'precision_H': array([nan]), 'recall_H': array([0.]), 'UEO': array([0.23447353]), 'f1_L': array([0.88470877]), 'f1_H': array([nan]), 'f1': 67.01}, 'metrics_audited': {'precision': array([0.90452009]), 'recall': array([0.9563582]), 'f1': array([0.92971712])}, 'exp': 196}}, {'other_uncertainty_metrics': {'sUEO': 0.0, 'ece_score': 0.01947295218764906}, 'uncertainty_result': {'metrics': {'precision_L': array([nan]), 'recall_L': array([0.]), 'recall_Ltotal': array([0.]), 'AA': array([0.03435077]), 'precision_H': array([nan]), 'recall_H': array([0.]), 'UEO': array([0.54008159]), 'f1_L': array([nan]), 'f1_H': array([nan]), 'f1': 0.0}, 'metrics_audited': {'precision': array([1.]), 'recall': array([0.96602314]), 'f1': array([0.98271797]

In [ ]:
print(len(results))

10


In [ ]:
results[9]

{'other_uncertainty_metrics': {'sUEO': 0.5404739587669327,
  'ece_score': 0.03641546979657238},
 'uncertainty_result': {'metrics': {'precision_L': array([0.92989761]),
   'recall_L': array([0.93764161]),
   'recall_Ltotal': array([0.503057]),
   'AA': array([0.02999948]),
   'precision_H': array([0.58242773]),
   'recall_H': array([0.18413217]),
   'UEO': array([0.26910476]),
   'f1_L': array([0.93375355]),
   'f1_H': array([0.27980509]),
   'f1': 69.74},
  'metrics_audited': {'precision': array([0.96224463]),
   'recall': array([0.9665439]),
   'f1': array([0.96438947])},
  'exp': 205}}

In [ ]:
trainer.f1

69.74

In [ ]:
print(trainer.f1)

69.74
